In [1]:
import os
import csv
import pickle
import random
import math
import dicom
import numpy as np
import scipy
from tqdm import tqdm
from natsort import natsorted
from skimage import transform
from sklearn.externals import joblib

import matplotlib.pyplot as plt
%matplotlib inline

#### load and shuffle data

In [2]:
(pts_train, pts_train_val, 
 data_train, label_sys_train, label_dia_train, 
 data_train_val, label_sys_train_val, label_dia_train_val, 
 data_val, data_val_pt_index) = joblib.load('../data_proc/2-data_processed.pkl')

In [3]:
shuffle_index = list(range(data_train.shape[0]))
random.shuffle(shuffle_index)
data_train = data_train[shuffle_index]
label_sys_train = label_sys_train[shuffle_index]
label_dia_train = label_dia_train[shuffle_index]

shuffle_index = list(range(data_train_val.shape[0]))
random.shuffle(shuffle_index)
data_train_val = data_train_val[shuffle_index]
label_sys_train_val = label_sys_train_val[shuffle_index]
label_dia_train_val = label_dia_train_val[shuffle_index]

#### training

In [4]:
from keras.models import Sequential, Graph
from keras.layers.core import Activation, Dense, Dropout, Flatten, Merge, Reshape, Lambda
from keras.layers.core import TimeDistributedDense, TimeDistributedMerge
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ParametricSoftplus, ELU
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianDropout, GaussianNoise
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using Theano backend.


Using gpu device 2: Tesla K80 (CNMeM is disabled)


Continuous Ranked Probability Score (used as loss function)

In [5]:
from keras import backend as K
from theano import tensor as T

# currently for theano, would need modifications for tensorflow

def CRPS(y_true, y_pred):
    return K.mean(K.square(T.cumsum(y_pred, axis=-1) - y_true), axis=-1)

In [6]:
from keras.layers.core import Layer
from keras.initializations import uniform
from keras import backend as K
from theano import tensor as T

# currently for theano, would need modifications for tensorflow

class DynamicWindow(Layer):
    
    def __init__(self, center_scale=0.1, width_scale=1.0, **kwargs):
        self.input = K.placeholder(ndim=3)
        self.center_scale = center_scale
        self.width_scale = width_scale
        super(DynamicWindow, self).__init__(**kwargs)

    def build(self):
        stack_size = self.input_shape[1]
        self.center = uniform((1,), scale=self.center_scale)
        self.width = uniform((1,), scale=self.width_scale)
        self.params = [self.center, self.width]

    def get_output(self, train=False):
        X = self.get_input(train)
        stack_size = self.input_shape[1]
        rows = self.input_shape[2]
        cols = self.input_shape[3]
        output = T.clip(X - T.reshape(T.repeat(self.center, stack_size*rows*cols, axis=0), (stack_size, rows, cols)), 
                        T.reshape(T.repeat(self.center - T.abs_(self.width / 2), stack_size*rows*cols, axis=0), (stack_size, rows, cols)), 
                        T.reshape(T.repeat(self.center + T.abs_(self.width / 2), stack_size*rows*cols, axis=0), (stack_size, rows, cols)))
        return output

    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(DynamicWindow, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [7]:
model_sys_branches = []
for i in range(5):
    branch = Sequential()
    branch.add(DynamicWindow(center_scale=0.1, width_scale=1.0, input_shape=(30, 250, 250)))
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'))
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
    branch.add(Flatten())
    model_sys_branches.append(branch)

model_sys = Sequential()
model_sys.add(Merge(model_sys_branches, mode='concat', concat_axis=-1))
model_sys.add(Dense(2048, activation='relu'))
model_sys.add(Dropout(0.5))
model_sys.add(Dense(600))
model_sys.add(Activation('softmax'))

model_sys.compile(loss=CRPS, optimizer='adam')

TypeError: add() got an unexpected keyword argument 'mode'

In [ ]:
batch_size = 8
nb_epoch = 100
    
checkpointer = ModelCheckpoint(filepath='../model_weights/2-1-convnet_multidynwin_systole.hdf5', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model_sys.fit([data_train]*5, label_sys_train, 
              batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=False, verbose=1,
              validation_data=([data_train_val]*5, label_sys_train_val), shuffle=True,
              callbacks=[checkpointer, earlystopping])

In [ ]:
model_dia_branches = []
for i in range(5):
    branch = Sequential()
    branch.add(DynamicWindow(center_scale=0.1, width_scale=1.0, input_shape=(30, 250, 250)))
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'))
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
    branch.add(BatchNormalization())
    branch.add(ELU())
    branch.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
    branch.add(Flatten())
    model_dia_branches.append(branch)

model_dia = Sequential()
model_dia.add(Merge(model_dia_branches, mode='concat', concat_axis=-1))
model_dia.add(Dense(2048, activation='relu'))
model_dia.add(Dropout(0.5))
model_dia.add(Dense(600))
model_dia.add(Activation('softmax'))

model_dia.compile(loss=CRPS, optimizer='adam')

In [ ]:
batch_size = 8
nb_epoch = 100
    
checkpointer = ModelCheckpoint(filepath='../model_weights/2-1-convnet_multidynwin_diastole.hdf5', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model_dia.fit([data_train]*5, label_dia_train, 
              batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=False, verbose=1,
              validation_data=([data_train_val]*5, label_dia_train_val), shuffle=True,
              callbacks=[checkpointer, earlystopping])

#### predict

In [ ]:
model_sys.load_weights('../model_weights/2-1-convnet_multidynwin_systole.hdf5')
model_dia.load_weights('../model_weights/2-1-convnet_multidynwin_diastole.hdf5')

preds_sys = model_sys.predict(data_val, verbose=0)
preds_dia = model_dia.predict(data_val, verbose=0)

preds_sys = np.clip(np.cumsum(preds_sys, axis=-1), 0, 1)
preds_dia = np.clip(np.cumsum(preds_dia, axis=-1), 0, 1)

In [ ]:
preds_sys_pt = {}
preds_dia_pt = {}
for pt in range(501, 701):
    preds_sys_pt[pt] = np.mean(preds_sys[np.where(data_val_pt_index == pt)[0]], axis=0)
    preds_dia_pt[pt] = np.mean(preds_dia[np.where(data_val_pt_index == pt)[0]], axis=0)

In [ ]:
with open('../data/sample_submission_validate.csv', 'r') as fi:
    reader = csv.reader(fi)
    header = next(reader)
    
    with open('../submissions/2-1-convnet_multidynwin.csv', 'w') as fo:
        writer = csv.writer(fo, lineterminator='\n')
        writer.writerow(header)
        for rowin in tqdm(reader):
            _id = rowin[0]
            pt, mode = _id.split('_')
            rowout = [_id]
            if mode.lower() == 'systole':
                rowout.extend(preds_sys_pt[int(pt)].tolist())
            elif mode.lower() == 'diastole':
                rowout.extend(preds_dia_pt[int(pt)].tolist())
            else:
                raise
            writer.writerow(rowout)